In [ ]:
import requests
import datetime
import time
import pandas as pd
import json
import boto3
import http.client

def lambda_handler(event, context):
    api_key = "30548bfbf8msh1742529d7115018p1ffd33jsnb2403e22cd90"
    league_id = 140
    season = '2023'
    conn = http.client.HTTPSConnection("api-football-v1.p.rapidapi.com")
    
    headers = {
        'x-rapidapi-host': "api-football-v1.p.rapidapi.com",
        'x-rapidapi-key': api_key
    }
    
    # Make the request to get the fixtures for La Liga in the 2023 season
    conn.request("GET", f"/v3/fixtures?league={league_id}&season={season}", headers=headers)
    
    # Get the response
    res = conn.getresponse()
    data = res.read()
    
    # Decode the response and parse it as JSON
    data_json = json.loads(data.decode("utf-8"))
    
    # Extract the fixtures from the JSON response
    fixtures = data_json['response']
    
    # extraxt the statistics for the fixture
    df = pd.DataFrame(fixtures)
    df['fixture_id'] = df['fixture'].apply(lambda x: x['id'])
    
    i = 0
    statistics_fixtures_all = []
    statistics_players_all = []
    for fixture_id in df['fixture_id']:
        i = i + 1
        headers = {
            'x-rapidapi-host': "api-football-v1.p.rapidapi.com",
            'x-rapidapi-key': api_key
        }
    
        conn.request("GET", f"/v3/fixtures/statistics?fixture={fixture_id}", headers=headers)
    
        res = conn.getresponse()
        data = res.read()
    
        statistics_fixtures_all.append(data.decode("utf-8"))
        
        headers = {
            'x-rapidapi-host': "api-football-v1.p.rapidapi.com",
            'x-rapidapi-key': api_key
        }
    
        conn.request("GET", f"/v3/fixtures/players?fixture={fixture_id}", headers=headers)
    
        res = conn.getresponse()
        data = res.read()
    
        statistics_players_all.append(data.decode("utf-8"))
        
        if i == 30:
            time.sleep(20)
            i = 0
    
    
    current_datetime = datetime.datetime.now()
    formatted_datetime = current_datetime.strftime("%Y-%m-%d %H:%M")
    
    # Upload to S3
    s3_bucket_name = "dwlprojectbucket"
    s3 = boto3.client('s3')
    
    # Upload fixtures to S3 with public read permissions
    s3_key = f"{formatted_datetime} - fixtures.json"
    try:
        s3.put_object(Bucket=s3_bucket_name, Key=s3_key, Body=json.dumps(fixtures), ACL='public-read')
    except Exception as e:
        print(f"Failed to upload fixtures data to S3: {e}")
        return {
            'statusCode': 500,
            'body': json.dumps('Failed to upload fixtures data to S3')
        }
        
    # Upload statistics_fixtures_all to S3 with public read permissions
    s3_key = f"{formatted_datetime} - statistics_fixtures_all.json"
    try:
        s3.put_object(Bucket=s3_bucket_name, Key=s3_key, Body=json.dumps(statistics_fixtures_all), ACL='public-read')
    except Exception as e:
        print(f"Failed to upload statistics_fixtures_all data to S3: {e}")
        return {
            'statusCode': 500,
            'body': json.dumps('Failed to upload statistics_fixtures_all data to S3')
        }
        
    # Upload statistics_players_all to S3 with public read permissions
    s3_key = f"{formatted_datetime} - statistics_players_all.json"
    try:
        s3.put_object(Bucket=s3_bucket_name, Key=s3_key, Body=json.dumps(statistics_players_all), ACL='public-read')
    except Exception as e:
        print(f"Failed to upload statistics_players_all data to S3: {e}")
        return {
            'statusCode': 500,
            'body': json.dumps('Failed to upload statistics_players_all data to S3')
        }
    
    return {
        'statusCode': 200,
        'body': json.dumps('Data successfully uploaded to S3 with public read permissions')
    }
